### Store Cleaned Data in PostgreSQL
* Insert cleaned review data using Python 

In [1]:
from dotenv import  load_dotenv
import psycopg2
import os

In [2]:
load_dotenv()

True

In [4]:
import pandas as pd
df=pd.read_csv("../data/processed/bank_reviews_with_themes.csv")
df.head(5)

,review_id,review,rating,date,bank,source,sentiment_label,sentiment_score,clean_review,topic,theme
0,5112423d-e618-44ba-ba49-62677cb76cd6,This application is very important and advanta...,5,2025-11-28,Commercial Bank of Ethiopia,Google Play Store,POSITIVE,0.998468,this application is very important and advanta...,1,General Praise / Mixed Feedback
1,bcb34681-1dd4-4781-b400-4393bb10b1d9,why didn't work this app?,1,2025-11-28,Commercial Bank of Ethiopia,Google Play Store,NEGATIVE,0.999132,why didnt work this app,1,General Praise / Mixed Feedback
2,c69f051a-00f8-4144-8423-b7ebcd328d2d,The app makes our life easier. Thank you CBE!,5,2025-11-28,Commercial Bank of Ethiopia,Google Play Store,POSITIVE,0.999696,the app makes our life easier thank you cbe,1,General Praise / Mixed Feedback
3,f8002d06-b5c5-4ed1-9d51-a9a379304cf8,the most advanced app. but how to stay safe?,5,2025-11-27,Commercial Bank of Ethiopia,Google Play Store,NEGATIVE,0.956510,the most advanced app but how to stay safe,1,General Praise / Mixed Feedback
4,81000db5-aa51-467e-826c-fc96160e96a8,Good application,4,2025-11-27,Commercial Bank of Ethiopia,Google Play Store,POSITIVE,0.999855,good application,1,General Praise / Mixed Feedback


In [7]:
# create bank_id mapping

bank_ids = {
    "Commercial Bank of Ethiopia":1,
    "Bank of Abyssinia":2,
    "Dashen Bank":3
}
df['bank_id']= df['bank'].map(bank_ids)
df.head(5)

,review_id,review,rating,date,bank,source,sentiment_label,sentiment_score,clean_review,topic,theme,bank_id
0,5112423d-e618-44ba-ba49-62677cb76cd6,This application is very important and advanta...,5,2025-11-28,Commercial Bank of Ethiopia,Google Play Store,POSITIVE,0.998468,this application is very important and advanta...,1,General Praise / Mixed Feedback,1
1,bcb34681-1dd4-4781-b400-4393bb10b1d9,why didn't work this app?,1,2025-11-28,Commercial Bank of Ethiopia,Google Play Store,NEGATIVE,0.999132,why didnt work this app,1,General Praise / Mixed Feedback,1
2,c69f051a-00f8-4144-8423-b7ebcd328d2d,The app makes our life easier. Thank you CBE!,5,2025-11-28,Commercial Bank of Ethiopia,Google Play Store,POSITIVE,0.999696,the app makes our life easier thank you cbe,1,General Praise / Mixed Feedback,1
3,f8002d06-b5c5-4ed1-9d51-a9a379304cf8,the most advanced app. but how to stay safe?,5,2025-11-27,Commercial Bank of Ethiopia,Google Play Store,NEGATIVE,0.956510,the most advanced app but how to stay safe,1,General Praise / Mixed Feedback,1
4,81000db5-aa51-467e-826c-fc96160e96a8,Good application,4,2025-11-27,Commercial Bank of Ethiopia,Google Play Store,POSITIVE,0.999855,good application,1,General Praise / Mixed Feedback,1


#### Insert cleaned review data using psycopg2

In [3]:
conn=psycopg2.connect(
    host="localhost",
    database="bank_reviews",
    user="postgres",
    password=os.getenv("DB_PASSWORD")
) #create a database connection
cursor=conn.cursor() #create a cursor object

In [28]:
# prepare the insert query
from psycopg2.extras import execute_batch
insert_query = """
     INSERT INTO review(
     review_id,
     bank_id,
     review,
     rating,
     date,
     sentiment_label,
     sentiment_score,
     source
     )
     VALUES (%s, %s, %s, %s, %s, %s, %s, %s)
     ON CONFLICT (review_id) DO NOTHING;
"""
# iterate over the DataFrame rows and Insert each row into the database
records=[(
    row.review_id,
    row.bank_id,
    row.review,
    row.rating,
    row.date,
    row.sentiment_label,
    row.sentiment_score,
    row.source
) for index, row in df.iterrows()
]
# execute the insert query for all records
execute_batch(cursor, insert_query, records,page_size=500)
# commit the changes and close the connection
conn.commit()
cursor.close()
conn.close()
print("Data inserted successfully")



InFailedSqlTransaction: current transaction is aborted, commands ignored until end of transaction block


In [29]:
conn.rollback()  # reset the failed transaction


In [13]:
print(df.isnull().sum())  # see if any columns have missing values


review_id          0
review             0
rating             0
date               0
bank               0
source             0
sentiment_label    0
sentiment_score    0
clean_review       0
topic              0
theme              0
bank_id            0
dtype: int64


In [30]:
conn.rollback()  # important before retrying

execute_batch(cursor, insert_query, records, page_size=500)
conn.commit()
cursor.close()